# 1. EDA
En este apartado se pretende analizar los datasets para poder enfocar mejor la limpieza

## 1.1 Importación y carga de datos
Debemos declarar las librerías que usamos y leer el correspondiente archivo de datos

In [278]:
# Importación de librerías
import pandas as pd

# Lectura dataset
df = pd.read_csv('../IncidenciasUsuariosSucio.csv')

## 1.2 Configuración de Pandas
Para poder leer bien los resultados de las ejecuciones, vamos a configurar tanto el número máximo de columnas como el número máximo de filas

In [279]:
# Número máximo de filas a mostrar
pd.set_option('display.max_rows', None)

# Número máximo de columnas a mostrar
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', 2000)

## 1.3 Descripción general del dataset
Para poder conocer ciertas características relevantes del dataset, como el número de instancias (filas) y características (columnas) procederemos a usar diferentes funciones de Pandas


In [280]:
# Descripción de parámetros generales -> count, mean, std, min, 25%, 50%, 75%, max
print(df.describe())

# Número de filas y columnas del dataset
print("\n")
print("Número de filas: ", df.shape[0])
print("Número de columnas: ", df.shape[1])
print("\n")

# Para saber el tipo de variable de cada columna
print(df.info())

                 ID
count  17633.000000
mean    8814.868485
std     5088.942963
min        1.000000
25%     4408.000000
50%     8816.000000
75%    13222.000000
max    17628.000000


Número de filas:  17633
Número de columnas:  6


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17633 entries, 0 to 17632
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   ID               17633 non-null  int64 
 1   TIPO_INCIDENCIA  17633 non-null  object
 2   FECHA_REPORTE    17633 non-null  object
 3   ESTADO           17633 non-null  object
 4   UsuarioID        17633 non-null  object
 5   MantenimeintoID  17633 non-null  object
dtypes: int64(1), object(5)
memory usage: 826.7+ KB
None


## 1.4 Observación inicial del dataset
Vamos a mostrar 30 entradas para poder observar cómo es realmente por dentro el dataset

In [281]:
# Mostramos las 10 primeras filas, 10 filas aleatorias y las 10 últimas
print("   ----------    10 primeras filas    ----------    ")
print("\n")
print(df.head(10))
print("\n")
print("   ----------    10 filas aleatorias    ----------    ")
print("\n")
print(df.sample(10))
print("\n")
print("   ----------    10 últimas filas    ----------    ")
print("\n")
print(df.tail(10))

   ----------    10 primeras filas    ----------    


   ID     TIPO_INCIDENCIA FECHA_REPORTE   ESTADO                                      UsuarioID             MantenimeintoID
0   1            Desgaste    04-06-2024  Cerrada                 ['877-15-7376', '467-34-2729']               ['MNT-13905']
1   2            Desgaste    18-10-2023  Abierta                 ['592-42-6016', '339-07-9203']               ['MNT-00687']
2   3              Rotura    27/07/2024  Cerrada                 ['012-65-0978', '976-02-8152']               ['MNT-00376']
3   4          Vandalismo    12-25-2023  Cerrada  ['588-11-9551', '532-78-5662', '166-45-6290']               ['MNT-02131']
4   5  Mal funcionamiento    01-14-2024  Cerrada                 ['004-60-5880', '563-73-1437']  ['MNT-05277', 'MNT-08362']
5   6  Mal funcionamiento    16-06-2024  Cerrada                                ['243-58-3707']               ['MNT-00817']
6   7  Mal funcionamiento    2023-11-29  Abierta                 ['568-97-143

## 1.5 Revisión de valores nulos
Como ya se ha visto en el anterior apartado (info), podemos observar los valores nulos de esa forma. Pero se puede observar de una forma más visual en esta sección y, además, hay que tener en cuenta valores como el cero que también pueden considerarse nulos.

In [282]:
# Para poder saber el número de valores faltantes
print("    ----------    Valores faltantes    ----------    ")
print("\n")
print(df.isnull().sum())
print("\n")

# Para poder saber el número de ceros en cada columna
print("    ----------    Valores cero    ----------    ")
print("\n")
print((df == 0).sum())

    ----------    Valores faltantes    ----------    


ID                 0
TIPO_INCIDENCIA    0
FECHA_REPORTE      0
ESTADO             0
UsuarioID          0
MantenimeintoID    0
dtype: int64


    ----------    Valores cero    ----------    


ID                 0
TIPO_INCIDENCIA    0
FECHA_REPORTE      0
ESTADO             0
UsuarioID          0
MantenimeintoID    0
dtype: int64


No existen valores nulos en este dataset, por lo que no se hará una limpieza de nulos

## 1.6 Identificación de fechas no estandarizadas
Se deben identificar las fechas que no se encuentran en el formato adecuado para MongoDB (DD/MM/YYYY)

In [283]:
# Vamos a mostrar algunas fechas para poder observar en qué formato están
print("   ----------    Fechas    ----------    ")
print("\n")
print(df['FECHA_REPORTE'].sample(20))

   ----------    Fechas    ----------    


4418     2024/07/05
3730     2024-07-08
1240     2023-12-06
14752    08-05-2024
8554     14/06/2024
9231     07-12-2023
4871     2024/05/02
5043     07-04-2024
4415     24-11-2023
17351    08/06/2024
2056     10-09-2024
11894    15/12/2023
6197     11-13-2023
1074     2023-10-25
14530    2024/01/24
5845     02-10-2024
2809     02/09/2024
15025    15-03-2024
15803    2024/04/15
1778     08-21-2024
Name: FECHA_REPORTE, dtype: object


Como se puede apreciar, las fechas se encuentran en diversos formatos que deben ser homogeneizados

## 1.7 Identificación de registros duplicados
Debemos validar que no existen filas iguales que ensucien el dataset, sobre todo estando pendiente de duplicaciones de la clave primaria

In [284]:
# Ver las filas duplicadas
print("   ----------    Filas duplicadas    ----------    ")
print("\n")
print(df[df.duplicated()])
print("\n")
# Número de filas duplicadas
print("Número de filas duplicadas: ", df.duplicated().sum())
print("\n")

# Filtrar filas que tienen el mismo ID
duplicados = df.groupby('ID').filter(lambda x: len(x) > 1)

# Ordenar por NIF para que las filas con el mismo NIF se visualicen una encima de la otra
duplicados = duplicados.sort_values(by='ID')

# Mostrar las filas con la misma PK
print("   ----------    Filas con el mismo ID    ----------    ")
print("\n")
print(duplicados)
print("\n")
print("Número de filas con el mismo ID: ", duplicados.shape[0])

   ----------    Filas duplicadas    ----------    


          ID TIPO_INCIDENCIA FECHA_REPORTE   ESTADO        UsuarioID MantenimeintoID
17630  12633      Vandalismo    02/12/2023  Cerrada  ['903-64-1048']   ['MNT-11152']


Número de filas duplicadas:  1


   ----------    Filas con el mismo ID    ----------    


          ID TIPO_INCIDENCIA FECHA_REPORTE   ESTADO                                      UsuarioID                          MantenimeintoID
430      431          Rotura    08-14-2024  Cerrada  ['122-92-6750', '293-31-6681', '599-62-2214']                            ['MNT-03964']
17628    431          Rotura    14/08/2024  Cerrada  ['122-92-6750', '293-31-6681', '599-62-2214']                            ['MNT-03964']
9947    9948        Desgaste    03-04-2024  Cerrada                                ['722-68-2700']                            ['MNT-14428']
17631   9948        Desgaste    04-03-2024  Cerrada                                ['722-68-2700']                        

Existen filas iguales que solo difieren en el formato de la fecha, por lo que se pueden considerar filas iguales que deberán ser eliminadas, ya que no aportan información adicional y solo ensucian el dataset

## 1.8 Búsqueda de errores tipográficos
Hay ciertos atributos de texto que pueden contar con determinados errores tipográficos que deben ser solucionados, como los nombres de áreas, juegos, usuarios y ubicaciones

En este dataset no encontramos ningún atributo similar, por lo que este paso no se realizará

## 1.9 Identificación de valores enum fuera de campo
Hay ciertos atributos que solo deben poseer ciertos valores (como Operativo-NoOperativo). Hace falta identificar aquellos valores de ese campo fuera de la norma

In [285]:
# Muestra todos los valores distintos de la columna 'TIPO_INCIDENCIA'
print("   ----------    Valores distintos de la columna 'TIPO_INCIDENCIA'    ----------    ")
print("\n")
print(df['TIPO_INCIDENCIA'].unique())
print("\n")

# Muestra todos los valores distintos de la columna 'ESTADO'
print("   ----------    Valores distintos de la columna 'ESTADO'    ----------    ")
print("\n")
print(df['ESTADO'].unique())

   ----------    Valores distintos de la columna 'TIPO_INCIDENCIA'    ----------    


['Desgaste' 'Rotura' 'Vandalismo' 'Mal funcionamiento']


   ----------    Valores distintos de la columna 'ESTADO'    ----------    


['Cerrada' 'Abierta']


Todos los Enums poseen valores acordes con lo esperado, por lo que no se va a realizar una limpieza de ellos.

## 1.10 Validación de las coordenadas y otros campos geoespaciales
Hay algunas veces en las que los códigos postales no respetan la identificación de Madrid (280..) o el formato, ya sean códigos postales u otros atributos de geolocalización

En este dataset no encontramos valores de localización, por lo que este paso no se realizará

## 1.11 Identificación de unidades de medida en un formato no estandarizado
Se deben identificar las filas que no posean un formato estándar

En este dataset no encontramos valores de unidades de medida, por lo que este paso no se realizará

## 1.12 Otros atributos a corregir
En esta sección se mencionarán aquellos atributos que también deban ser limpiados por errores

### 1.12.1 Búsqueda de IDs

In [286]:
# Vamos a mostrar 10 IDs para ver cómo se encuentran
print("   ----------    IDs    ----------    ")
print("\n")
print(df['ID'].sample(10))

   ----------    IDs    ----------    


14385    14386
2659      2660
15509    15510
4664      4665
2488      2489
15441    15442
2169      2170
16255    16256
14009    14010
8185      8186
Name: ID, dtype: int64


Vamos a validar que los IDs son números enteros

In [287]:
# Validar que todos los valores de ID sean enteros
print("   ----------    Validación de ID    ----------    ")
print("\n")
print(df['ID'].apply(lambda x: x.is_integer()).all())

   ----------    Validación de ID    ----------    


True


Por ende, todos los IDs son válidos y no se deberán limpiar

### 1.12.2 Validar todos los valores de la columna UsuarioID y MantenimientoID
Hay que comprobar que todos los valores de estos campos siguen el formato de los ID de usuarios (XXX-XX-XXXX) sacado de Usuarios y de los IDs de mantenimiento, sacados de Mantenimiento. Además, debemos validar que todos los valores en ambas columnas están presentes en los otros datasets, para no estar referenciando a un valor que no existe

In [288]:
# Extraemos todos los valores del campo de 'UsuarioID' a una lista
# Como todos están en formato string aunque parezcan listas, solo debemos eliminar los corchetes y las comillas y comas para colocarlo en una lista
usuarios = df['UsuarioID'].tolist()
usuarios = [x.replace('[', '').replace(']', '').replace(', ', '|').replace("'", '') for x in usuarios]
valido = True
for elem in usuarios:
    # Si el elemento posee un | hacer un split del |:
    if '|' in elem:
        lista = elem.split('|')
        for e in lista:
            # Si el elemento no tiene el formato "XXX-XX-XXXX", siendo X un número, entonces devuelve False
            if not e[0:3].isdigit() or not e[4:6].isdigit() or not e[7:].isdigit(): 
                valido = False
    else:
        if not e[0:3].isdigit() or not e[4:6].isdigit() or not e[7:].isdigit():
            valido = False
print("Todos los valores son IDs válidos de Usuarios: ", valido)

# Extraemos todos los valores del campo de 'MantenimientoID' a una lista
# Como todos están en formato string aunque parezcan listas, solo debemos eliminar los corchetes y las comillas y comas para colocarlo en una lista
mantenimiento = df['MantenimeintoID'].tolist()
mantenimiento = [x.replace('[', '').replace(']', '').replace(', ', '|').replace("'", '') for x in mantenimiento]
valido = True
for elem in mantenimiento:
    # Si el elemento posee un | hacer un split del |:
    if '|' in elem:
        lista = elem.split('|')
        for e in lista:
            # Si el elemento no tiene el formato "MNT-XXXXX", siendo X un número, entonces devuelve False
            if not e[0:4] == 'MNT-' or not e[4:].isdigit():
                valido = False
    else:
        if not elem[0:4] == 'MNT-' or not elem[4:].isdigit():
            print(elem)
            print(elem[0:4])
            print(elem[4:])
            valido = False
print("Todos los valores son IDs válidos de Mantenimiento: ", valido)
print("\n")

usuariosbis = df['UsuarioID'].tolist()
usuarios_extraidos = []
# Pasamos las listas de listas que se encuentren dentro de la lista a elementos individuales
for elem in usuariosbis:
    elem = elem.replace('[', '').replace(']', '').replace(', ', '|').replace("'", '')
    if '|' in elem:
        lista = elem.split('|')
        for e in lista:
            usuarios_extraidos.append(e)
    else:
        usuarios_extraidos.append(elem)

mantenimientobis = df['MantenimeintoID'].tolist()
mantenimiento_extraidos = []
# Pasamos las listas de listas que se encuentren dentro de la lista a elementos individuales
for elem in mantenimientobis:
    elem = elem.replace('[', '').replace(']', '').replace(', ', '|').replace("'", '')
    if '|' in elem:
        lista = elem.split('|')
        for e in lista:
            mantenimiento_extraidos.append(e)
    else:
        mantenimiento_extraidos.append(elem)

# Validar que todos los valores de UsuarioID y MantenimientoID se encuentren en UsuariosLimpio.csv y MantenimientoLimpio.csv
usuarios = pd.read_csv('../UsuariosLimpio.csv')
mantenimiento = pd.read_csv('../MantenimientoLimpio.csv')
valido = True
for elem in usuarios_extraidos:
    if elem not in usuarios['NIF'].tolist():
        valido = False
for elem in mantenimiento_extraidos:
    if elem not in mantenimiento['ID'].tolist():
        valido = False
print("Todos los valores de UsuarioID y MantenimientoID se encuentran en los datasets correspondientes: ", valido)




Todos los valores son IDs válidos de Usuarios:  True
Todos los valores son IDs válidos de Mantenimiento:  True


Todos los valores de UsuarioID y MantenimientoID se encuentran en los datasets correspondientes:  True


Gracias al script anterior se puede observar como todos los IDs son válidos, tanto de mantenimiento como de usuario

### 1.12.3 Consideraciones extras de Fechas
Cabe recalcar que si el campo de fechas no posee un formato esperado, las funciones de limpieza darán error, por lo que no hace falta hacer ahora las comprobaciones de sus valores para determinar si son correctos.

### 1.12.4 Cambio de nombre columna de "MantenimeintoID" a "MantenimientoID"
Esa columna posee una errata en su nombre que debe ser solucionada 

In [289]:
# Cambiar nombre de la columna "MantenimeintoID" a "MantenimientoID"
df.rename(columns={'MantenimeintoID': 'MantenimientoID'}, inplace=True)

# Mostramos el resultado del cambio con una muestra de 1 fila
print(df.sample(1))

          ID TIPO_INCIDENCIA FECHA_REPORTE   ESTADO        UsuarioID             MantenimientoID
17166  17167          Rotura    18-05-2024  Abierta  ['452-57-5193']  ['MNT-05293', 'MNT-08831']


Como se puede observar, el nombre de la columna ha cambiado

# 2. Limpieza de los datasets
En este apartado se realizará la limpieza según la información obtenida en el análisis exploratorio de datos:
- Se deben eliminar las filas repetidas que no aportan más información
- Se deben corregir las fechas y dejarlas en un formato estándar

## 2.1 Limpieza de filas repetidas

Debemos eliminar las filas repetidas que no aportan información nueva. Para ello se va a eliminar una de las dos repetidas indistintivamente, ya que solo varían en el formato de la fecha, el resto de datos son iguales (incluso la fecha, solo que se encuentra en distinto formato y por eso Pandas lo reconoce como distinta)

In [290]:
# Filtrar filas que tienen el mismo ID
duplicados = df.groupby('ID').filter(lambda x: len(x) > 1)

# Ordenar por NIF para que las filas con el mismo ID se visualicen una encima de la otra
duplicados = duplicados.sort_values(by='ID')

# Mostrar las filas con la misma PK
print("   ----------    Filas con el mismo ID    ----------    ")
print("\n")
print(duplicados)
print("\n")
print("Número de filas con el mismo ID: ", duplicados.shape[0])
print("\n")

# Limpiamos una de las dos filas con el mismo ID de duplicados
df.drop_duplicates(subset='ID', keep='first', inplace=True)

# Mostramos el resultado por pantalla
print("   ----------    Filas con el mismo ID después de limpiar    ----------    ")
print("\n")
print(df[df.duplicated()])
print("\n")
# Número de filas duplicadas
print("Número de filas duplicadas: ", df.duplicated().sum())
print("\n")

# Mostramos, para comprobar, 2 filas de las que antes estaban duplicadas
print("   ----------    2 filas de las que antes estaban duplicadas    ----------    ")
print("\n")
print(df.loc[df['ID'] == 431])
print(df.loc[df['ID'] == 9948])
print("\n")

# Mostramos el intento de acceder a una fila eliminada
print("   ----------    Intento de acceder a una fila eliminada    ----------    ")
print("\n")
# Accedemos a la fila con ID 431 y FECHA_REPORTE 14/08/2024
print(df.loc[(df['ID'] == 431) & (df['FECHA_REPORTE'] == '14/08/2024')])

   ----------    Filas con el mismo ID    ----------    


          ID TIPO_INCIDENCIA FECHA_REPORTE   ESTADO                                      UsuarioID                          MantenimientoID
430      431          Rotura    08-14-2024  Cerrada  ['122-92-6750', '293-31-6681', '599-62-2214']                            ['MNT-03964']
17628    431          Rotura    14/08/2024  Cerrada  ['122-92-6750', '293-31-6681', '599-62-2214']                            ['MNT-03964']
9947    9948        Desgaste    03-04-2024  Cerrada                                ['722-68-2700']                            ['MNT-14428']
17631   9948        Desgaste    04-03-2024  Cerrada                                ['722-68-2700']                            ['MNT-14428']
17630  12633      Vandalismo    02/12/2023  Cerrada                                ['903-64-1048']                            ['MNT-11152']
12632  12633      Vandalismo    02/12/2023  Cerrada                                ['903-64-1048']   

Se puede apreciar que ya no hay filas duplicadas y que, si intentamos acceder a una fila de las repetidas y eliminadas, no podemos

## 2.2 Limpieza de FECHA_REPORTE

Debemos dejar las fechas en un formato estándar lejible por MongoDB, como 'DD/MM/YYYY'

In [291]:
# Leemos toda la columna de fechas y almacenamos todas las fechas en una lista
fechas = df['FECHA_REPORTE'].tolist()

# Ahora, pasamos por todas las fechas y corregimos las fechas que están mal escritas
fechas_corregidas = []
año_al_final = False

for fecha in fechas:
    # Si fecha contiene un guion, hacer fecha.split('-') y si fecha contiene una barra, hacer fecha.split('/')
    if '-' in fecha:
        fecha_split = fecha.split('-')
    elif '/' in fecha:
        fecha_split = fecha.split('/')
    # Condiciones para saber el día, mes y año
    # Encontrar el año
    if (int(fecha_split[0])>31):
        año = fecha_split[0]
        año_al_final = False
    elif (int(fecha_split[2])>31):
        año = fecha_split[2]
        año_al_final = True
    else:
        print("No se ha encontrado el año")
        fechas_corregidas.append("fecha incorrecta")
        break
    # Encontrar el día y el mes
    # Si el año está al final, no comprobamos el primer caracter
    if año_al_final:
        if (int(fecha_split[0])<32 and int(fecha_split[0])>12):
            dia = fecha_split[0]
            mes = fecha_split[1]
        elif (int(fecha_split[1])<32 and int(fecha_split[1])>12):
            dia = fecha_split[1]
            mes = fecha_split[0]
        # Si no hay ningun número entre el 13 y el 31, se asume que el mes es el segundo siempre y el día es el primero
        else:
            dia = fecha_split[0]
            mes = fecha_split[1]
    else:
        if (int(fecha_split[1])<32 and int(fecha_split[1])>12):
            dia = fecha_split[1]
            mes = fecha_split[2]
        elif (int(fecha_split[2])<32 and int(fecha_split[2])>12):
            dia = fecha_split[2]
            mes = fecha_split[1]
        # Si no hay ningun número entre el 13 y el 31, se asume que el mes es el segundo siempre y el día es el tercero
        else:
            dia = fecha_split[2]
            mes = fecha_split[1]
    fechas_corregidas.append(dia+"/"+mes+"/"+año)
    
# Ahora, cambiamos toda la columna de FECHAS_INTERVENCION por las fechas corregidas
df['FECHA_REPORTE'] = fechas_corregidas

# Mostramos las fechas corregidas
print("   ----------    Columna de fechas corregidas    ----------    ")
print("\n")
print(df['FECHA_REPORTE'].sample(20))

   ----------    Columna de fechas corregidas    ----------    


1942     18/10/2023
16577    08/11/2023
1689     08/03/2024
7144     03/12/2023
12855    03/08/2024
14459    07/10/2024
14223    03/10/2024
14276    02/04/2024
7777     11/03/2024
5343     28/02/2024
9006     21/12/2023
13286    27/06/2024
11428    03/05/2024
7886     25/06/2024
15376    13/07/2024
10257    02/08/2024
3400     23/03/2024
16439    30/08/2024
2517     09/01/2024
8582     23/10/2023
Name: FECHA_REPORTE, dtype: object


Como se puede apreciar, las fechas ya están estandarizadas

## 2.4 Mostrar Dataset Limpio y guardar CSV
Vamos a mostrar algunas filas del dataset limpio para validar que todo está OK y guardamos el Dataset

In [292]:
# Enseñamos 20 filas aleatorias para ver cómo quedan
print("   ----------    10 filas aleatorias    ----------    ")
print("\n")
print(df.sample(10))
print("\n")

# Guardamos el dataset limpio
df.to_csv('../IncidenciasUsuariosLimpio.csv', index=False)
print("\n")
print("Dataset limpio guardado")

   ----------    10 filas aleatorias    ----------    


          ID     TIPO_INCIDENCIA FECHA_REPORTE   ESTADO                                      UsuarioID                          MantenimientoID
3453    3454            Desgaste    24/05/2024  Abierta                                ['591-78-5932']  ['MNT-02492', 'MNT-03711', 'MNT-11463']
4364    4365              Rotura    23/07/2024  Cerrada  ['849-79-8851', '948-68-9260', '218-02-7266']                            ['MNT-10359']
418      419            Desgaste    25/10/2023  Cerrada  ['764-34-5923', '564-49-2033', '060-85-1683']               ['MNT-08190', 'MNT-09211']
15524  15525  Mal funcionamiento    08/01/2024  Abierta                                ['635-29-6405']                            ['MNT-11152']
6680    6681              Rotura    22/05/2024  Abierta                                ['157-60-9878']                            ['MNT-10386']
3499    3500              Rotura    21/10/2023  Cerrada                        